# Examen Final: Redes Neuronales y Aprendizaje Profundo

## Generar una versión personalizada de traductor, Modelo para generación de su voz traducida en tiempo real al Inglés.

### Data Data: https://tatoeba.org/es/downloads

### 1) Instalando librerias requeridas 

In [44]:
pip install torch torchvision torchaudio transformers librosa sounddevice

Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [46]:
pip install SpeechRecognition

Note: you may need to restart the kernel to use updated packages.


In [47]:
pip install sacremoses

Note: you may need to restart the kernel to use updated packages.


In [48]:
pip install gtts

Note: you may need to restart the kernel to use updated packages.


### 2) Implementando el modelo con Transformer

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from collections import Counter
import math
import numpy as np
import re

torch.manual_seed(23)

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [8]:
##MAX_SEQ_LEN = 128
MAX_SEQ_LEN = 64

In [9]:
class PositionalEmbedding(nn.Module):
    def __init__(self, d_model, max_seq_len = MAX_SEQ_LEN):
        super().__init__()
        self.pos_embed_matrix = torch.zeros(max_seq_len, d_model, device=device)
        token_pos = torch.arange(0, max_seq_len, dtype = torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() 
                             * (-math.log(10000.0)/d_model))
        self.pos_embed_matrix[:, 0::2] = torch.sin(token_pos * div_term)
        self.pos_embed_matrix[:, 1::2] = torch.cos(token_pos * div_term)
        self.pos_embed_matrix = self.pos_embed_matrix.unsqueeze(0).transpose(0,1)
        
    def forward(self, x):
#         print(self.pos_embed_matrix.shape)
#         print(x.shape)
        return x + self.pos_embed_matrix[:x.size(0), :]

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model = 512, num_heads = 8):
        super().__init__()
        assert d_model % num_heads == 0, 'Embedding size not compatible with num heads'
        
        self.d_v = d_model // num_heads
        self.d_k = self.d_v
        self.num_heads = num_heads
        
        self.W_q = nn.Linear(d_model, d_model) ## query
        self.W_k = nn.Linear(d_model, d_model) ## Key
        self.W_v = nn.Linear(d_model, d_model) ## value
        self.W_o = nn.Linear(d_model, d_model)
        
    def forward(self, Q, K, V, mask = None):
        batch_size = Q.size(0)
        '''
        Q, K, V -> [batch_size, seq_len, num_heads*d_k]
        after transpose Q, K, V -> [batch_size, num_heads, seq_len, d_k]
        '''
        Q = self.W_q(Q).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2 )
        K = self.W_k(K).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2 )
        V = self.W_v(V).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2 )
        
        weighted_values, attention = self.scale_dot_product(Q, K, V, mask)
        weighted_values = weighted_values.transpose(1, 2).contiguous().view(batch_size, -1, self.num_heads*self.d_k)
        weighted_values = self.W_o(weighted_values)
        
        return weighted_values, attention
        
        
    def scale_dot_product(self, Q, K, V, mask = None):
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        attention = F.softmax(scores, dim = -1)
        weighted_values = torch.matmul(attention, V)
        
        return weighted_values, attention
        

class PositionFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        self.linear1 = nn.Linear(d_model, d_ff)
        self.linear2 = nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        return self.linear2(F.relu(self.linear1(x)))
    
class EncoderSubLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout = 0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.droupout1 = nn.Dropout(dropout)
        self.droupout2 = nn.Dropout(dropout)
    
    def forward(self, x, mask = None):
        attention_score, _ = self.self_attn(x, x, x, mask)
        x = x + self.droupout1(attention_score)
        x = self.norm1(x)
        x = x + self.droupout2(self.ffn(x))
        return self.norm2(x)

class Encoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([EncoderSubLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)
    def forward(self, x, mask=None):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

class DecoderSubLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super().__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)
        
    def forward(self, x, encoder_output, target_mask=None, encoder_mask=None):
        attention_score, _ = self.self_attn(x, x, x, target_mask)
        x = x + self.dropout1(attention_score)
        x = self.norm1(x)
        
        encoder_attn, _ = self.cross_attn(x, encoder_output, encoder_output, encoder_mask)
        x = x + self.dropout2(encoder_attn)
        x = self.norm2(x)
        
        ff_output = self.feed_forward(x)
        x = x + self.dropout3(ff_output)
        return self.norm3(x)
        
class Decoder(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([DecoderSubLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)
        
    def forward(self, x, encoder_output, target_mask, encoder_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, target_mask, encoder_mask)
        return self.norm(x)

In [10]:
class Transformer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, num_layers,
                 input_vocab_size, target_vocab_size, 
                 max_len=MAX_SEQ_LEN, dropout=0.1):
        super().__init__()
        self.encoder_embedding = nn.Embedding(input_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(target_vocab_size, d_model)
        self.pos_embedding = PositionalEmbedding(d_model, max_len)
        self.encoder = Encoder(d_model, num_heads, d_ff, num_layers, dropout)
        self.decoder = Decoder(d_model, num_heads, d_ff, num_layers, dropout)
        self.output_layer = nn.Linear(d_model, target_vocab_size)
        
    def forward(self, source, target):
        # Encoder mask
        source_mask, target_mask = self.mask(source, target)
        # Embedding and positional Encoding
        source = self.encoder_embedding(source) * math.sqrt(self.encoder_embedding.embedding_dim)
        source = self.pos_embedding(source)
        # Encoder
        encoder_output = self.encoder(source, source_mask)
        
        # Decoder embedding and postional encoding
        target = self.decoder_embedding(target) * math.sqrt(self.decoder_embedding.embedding_dim)
        target = self.pos_embedding(target)
        # Decoder
        output = self.decoder(target, encoder_output, target_mask, source_mask)
        
        return self.output_layer(output)
        
        
    
    def mask(self, source, target):
        source_mask = (source != 0).unsqueeze(1).unsqueeze(2)
        target_mask = (target != 0).unsqueeze(1).unsqueeze(2)
        size = target.size(1)
        no_mask = torch.tril(torch.ones((1, size, size), device=device)).bool()
        target_mask = target_mask & no_mask
        return source_mask, target_mask
        

### 3) Definiendo la data

In [11]:
## Archivo donde se guarda el diccionario descargado
PATH = "D:/Jose/UNI/Maestria IA/Cursos/03_Tercer_ciclo/03_Proyecto_Tesis/Proyecto/Modelo/Local/Transformer/corpus_total_formateado.txt"

In [12]:
## Identificando el archivo, separado por espacio en blanco
with open(PATH, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    
eng_spa_pairs = [line.strip().split('\t') for line in lines if line.count('\t') == 3]
##eng_spa_pairs = [line.strip().split('\t') for line in lines if '\t' in line]
##eng_spa_pairs = [line.strip().split('\t') for line in lines if line.count('\t') == 1]

In [13]:
eng_spa_pairs=eng_spa_pairs[1:]

In [14]:
eng_spa_pairs[:10]

[['000002',
  'b) ¿Cómo lograban mantenerse estos siervos de tiempo completo?',
  '000002',
  '(2) ¿Imaynatam tukuy tiemponkuwan Diosta serviq punta cristianokunaqa mantienekuqku?'],
 ['000003',
  'Cabe señalar que solo la Biblia explica de manera satisfactoria cuál es el origen de los muchos idiomas que conocemos hoy.',
  '000003',
  'Bibliallam allintapuni willawanchik imanasqam runakunaqa kunan tiempopi achka rimayniyoq kasqankuta.'],
 ['000004',
  'Pero al mismo tiempo nos advierte que “el amor al dinero es raíz de toda clase de males ”.',
  '000004',
  'Ichaqa nintaqmi: “Qollqella kuyayqa tukuy mana allin ruraykunapa mamanmi ”, nispa.'],
 ['000005',
  'Hoy, todo el que desea vivir de acuerdo con sus mandamientos considera que conmemorar el aniversario de la muerte de Cristo es de suma importancia.',
  '000005',
  'Kay tiempopipas Cristopa kamachisqanman hina kawsaqkunaqa ancha valorniyoqtam qawanku paypa wañukusqan punchaw yuyariytaqa.'],
 ['000006',
  'RESPUESTA: El Reino de Dios

In [15]:
eng_sentences = [pair[1] for pair in eng_spa_pairs]  ## índices de la ubicación del archivo descargado
spa_sentences = [pair[3] for pair in eng_spa_pairs]  ## índices de la ubicación del archivo descargado

In [16]:
print(eng_sentences[:10])
print(spa_sentences[:10])


['b) ¿Cómo lograban mantenerse estos siervos de tiempo completo?', 'Cabe señalar que solo la Biblia explica de manera satisfactoria cuál es el origen de los muchos idiomas que conocemos hoy.', 'Pero al mismo tiempo nos advierte que “el amor al dinero es raíz de toda clase de males ”.', 'Hoy, todo el que desea vivir de acuerdo con sus mandamientos considera que conmemorar el aniversario de la muerte de Cristo es de suma importancia.', 'RESPUESTA: El Reino de Dios es un gobierno celestial, y su Rey es Jesús.', '¿CUÁL ES LA SOLUCIÓN?', 'Adán y Eva desobedecieron a Dios, de modo que fueron expulsados del Edén.', 'Sin embargo, poco después de escribir esa carta, los de la casa de Cloe le informaron de que en la congregación de Corinto había graves divisiones.', 'Jesús cumplió de forma sorprendente esta profecía durante su ministerio.', 'Cierto diccionario bíblico lo define como “fortaleza espiritual, emocional y moral para hablar y actuar sin temor ante dificultades y peligros ” (The New In

In [17]:
## Limpiando las tildes
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[á]+", "a", sentence)
    sentence = re.sub(r"[é]+", "e", sentence)
    sentence = re.sub(r"[í]+", "i", sentence)
    sentence = re.sub(r"[ó]+", "o", sentence)
    sentence = re.sub(r"[ú]+", "u", sentence)
    sentence = re.sub(r"[^a-z]+", " ", sentence)
    sentence = sentence.strip()
    sentence = '<sos> ' + sentence + ' <eos>'
    return sentence

In [18]:
s1 = '¿Hola @ cómo estás? 123'

In [19]:
print(s1)
print(preprocess_sentence(s1))

¿Hola @ cómo estás? 123
<sos> hola como estas <eos>


In [20]:
eng_sentences = [preprocess_sentence(sentence) for sentence in eng_sentences]
spa_sentences = [preprocess_sentence(sentence) for sentence in spa_sentences]

In [21]:
spa_sentences[:10]

['<sos> imaynatam tukuy tiemponkuwan diosta serviq punta cristianokunaqa mantienekuqku <eos>',
 '<sos> bibliallam allintapuni willawanchik imanasqam runakunaqa kunan tiempopi achka rimayniyoq kasqankuta <eos>',
 '<sos> ichaqa nintaqmi qollqella kuyayqa tukuy mana allin ruraykunapa mamanmi nispa <eos>',
 '<sos> kay tiempopipas cristopa kamachisqanman hina kawsaqkunaqa ancha valorniyoqtam qawanku paypa wa ukusqan punchaw yuyariytaqa <eos>',
 '<sos> kutichiynin hanaq pachapi kaq huk gobiernom kamachiqnin ataqmi jesus <eos>',
 '<sos> imatam rurachwan <eos>',
 '<sos> adanwan evaqa jehova diostam mana kasukurqakuchu hinaspam edenmanta qarqochikurqaku <eos>',
 '<sos> ichaqa chay carta qellqasqan qepamanmi cloepa familian pabloman willaykurqaku corinto congregacionpi llumpay liryanakuy kasqanmanta <eos>',
 '<sos> jesusqa isaiaspa nisqantam kay pachapi diospa munayninta ruraspan allinta cumplirqa <eos>',
 '<sos> qarinchakuyqa llumpay sasachakuypi tarikuspa mana manchakuspa ruwaymi hinaspa mana 

### 4) Diccionario

In [22]:
## Diccionario (índices a palabras y palabras a índices)
def build_vocab(sentences):
    words = [word for sentence in sentences for word in sentence.split()]
    word_count = Counter(words)
    sorted_word_counts = sorted(word_count.items(), key=lambda x:x[1], reverse=True)  ## Ordenamiento de las palabras desc x nro. palabras
    word2idx = {word: idx for idx, (word, _) in enumerate(sorted_word_counts, 2)}
    word2idx['<pad>'] = 1   ## índices español de la ubicación del archivo descargado
    word2idx['<unk>'] = 3   ## índices ingles de la ubicación del archivo descargado
    idx2word = {idx: word for word, idx in word2idx.items()}
    return word2idx, idx2word

In [23]:
eng_word2idx, eng_idx2word = build_vocab(eng_sentences)
spa_word2idx, spa_idx2word = build_vocab(spa_sentences)
eng_vocab_size = len(eng_word2idx)  # Cantidad de enunciados
spa_vocab_size = len(spa_word2idx)  # Cantidad de enunciados

In [24]:
print(eng_vocab_size, spa_vocab_size)  ## Nro.de tokens

55320 234316


In [25]:
## Clase Dataset traducción
class EngSpaDataset(Dataset):
    def __init__(self, eng_sentences, spa_sentences, eng_word2idx, spa_word2idx):
        self.eng_sentences = eng_sentences  ## sentencia español
        self.spa_sentences = spa_sentences  ## sentencia ingles 
        self.eng_word2idx = eng_word2idx
        self.spa_word2idx = spa_word2idx
        
    def __len__(self):   ## tamaño del dataset
        return len(self.eng_sentences)
    
    def __getitem__(self, idx):  ## indexar cada elemento del dataset
        eng_sentence = self.eng_sentences[idx]
        spa_sentence = self.spa_sentences[idx]
        # return tokens idxs
        eng_idxs = [self.eng_word2idx.get(word, self.eng_word2idx['<unk>']) for word in eng_sentence.split()]
        spa_idxs = [self.spa_word2idx.get(word, self.spa_word2idx['<unk>']) for word in spa_sentence.split()]
        
        return torch.tensor(eng_idxs), torch.tensor(spa_idxs)

In [26]:
## Para el padding
def collate_fn(batch):
    eng_batch, spa_batch = zip(*batch)
    eng_batch = [seq[:MAX_SEQ_LEN].clone().detach() for seq in eng_batch]
    spa_batch = [seq[:MAX_SEQ_LEN].clone().detach() for seq in spa_batch]
    #eng_batch = torch.nn.utils.rnn.pad_sequence(eng_batch, batch_first=True, padding_value=0) 
    eng_batch = torch.nn.utils.rnn.pad_sequence(eng_batch, batch_first=True, padding_value=1) ## el padding viene de build_vocab
    spa_batch = torch.nn.utils.rnn.pad_sequence(spa_batch, batch_first=True, padding_value=1)
    return eng_batch, spa_batch
    

### 5) Entrenamiento del modelo (No se entrena)

In [28]:
## Entrenamiento del modelo
def train(model, dataloader, loss_function, optimiser, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0 
        for i, (eng_batch, spa_batch) in enumerate(dataloader):
            eng_batch = eng_batch.to(device)
            spa_batch = spa_batch.to(device)
            # Decoder preprocessing
            target_input = spa_batch[:, :-1]
            target_output = spa_batch[:, 1:].contiguous().view(-1)
            # Zero grads
            optimiser.zero_grad()
            # run model
            output = model(eng_batch, target_input)
            output = output.view(-1, output.size(-1))
            # loss\
            loss = loss_function(output, target_output)
            # gradient and update parameters
            loss.backward()
            optimiser.step()
            total_loss += loss.item()
            
        avg_loss = total_loss/len(dataloader)
        print(f'Epoch: {epoch}/{epochs}, Loss: {avg_loss:.4f}')
            
            

In [29]:
##BATCH_SIZE = 64
BATCH_SIZE = 32
dataset = EngSpaDataset(eng_sentences, spa_sentences, eng_word2idx, spa_word2idx)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

In [30]:
"""model = Transformer(d_model=512, num_heads=8, d_ff=2048, num_layers=6,
                    input_vocab_size=eng_vocab_size, target_vocab_size=spa_vocab_size,
                    max_len=MAX_SEQ_LEN, dropout=0.1)
"""
model = Transformer(d_model=128, num_heads=4, d_ff=512, num_layers=4,
                    input_vocab_size=eng_vocab_size, target_vocab_size=spa_vocab_size,
                    max_len=MAX_SEQ_LEN, dropout=0.1)


In [31]:
model = model.to(device)
loss_function = nn.CrossEntropyLoss(ignore_index=0)
optimiser = optim.Adam(model.parameters(), lr=0.0001)


In [32]:
## Guardando el modelo

In [33]:
#print(type(model))

In [34]:
#torch.save(model.state_dict(), "mi_modelo.pt")

In [35]:
## para cargar el modelo
#cmodel.load_state_dict(torch.load("D:/Jose/UNI/Maestria IA/Cursos/03_Tercer_ciclo/03_Proyecto_Tesis/Proyecto/Modelo/mi_modelo.pt")) ## carga en GPU pero solo cuando es diccionario
##model.load_state_dict(torch.load("D:/Jose/UNI/Maestria IA/Cursos/03_Tercer_ciclo/03_Proyecto_Tesis/Proyecto/Modelo/mi_modelo.pt",map_location=torch.device("cpu")))  ## Guarda en CPU 
##model.eval()
model = torch.load("D:/Jose/UNI/Maestria IA/Cursos/03_Tercer_ciclo/03_Proyecto_Tesis/Proyecto/Modelo/Local/Transformer/mi_modelo.pt", map_location="cpu", weights_only=False) ## cuando se sube el modelo completo
model.eval()

Transformer(
  (encoder_embedding): Embedding(15369, 512)
  (decoder_embedding): Embedding(10312, 512)
  (pos_embedding): PositionalEmbedding()
  (encoder): Encoder(
    (layers): ModuleList(
      (0-5): 6 x EncoderSubLayer(
        (self_attn): MultiHeadAttention(
          (W_q): Linear(in_features=512, out_features=512, bias=True)
          (W_k): Linear(in_features=512, out_features=512, bias=True)
          (W_v): Linear(in_features=512, out_features=512, bias=True)
          (W_o): Linear(in_features=512, out_features=512, bias=True)
        )
        (ffn): PositionFeedForward(
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
        )
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (droupout1): Dropout(p=0.1, inplace=False)
        (droupout2): Dropout(p=0.1, inplace=False)
   

### 6) Definiendo las funciones de traducción del modelo

In [36]:
def sentence_to_indices(sentence, word2idx):
    return [word2idx.get(word, word2idx['<unk>']) for word in sentence.split()]

def indices_to_sentence(indices, idx2word):
    filtered_indices = [idx for idx in indices if idx not in (spa_word2idx['<sos>'], spa_word2idx['<unk>'])]
    ## return ' '.join([idx2word[idx] for idx in indices if idx in idx2word and idx2word[idx] != '<pad>'])
    return ' '.join(spa_idx2word[idx] for idx in filtered_indices if idx in spa_idx2word)

def translate_sentence(model, sentence, eng_word2idx, spa_idx2word, max_len=MAX_SEQ_LEN, device='cpu'):
    model.eval()
    sentence = preprocess_sentence(sentence)
    input_indices = sentence_to_indices(sentence, eng_word2idx)
    input_tensor = torch.tensor(input_indices).unsqueeze(0).to(device)

    # Initialize the target tensor with <sos> token
    tgt_indices = [spa_word2idx['<sos>']]
    tgt_tensor = torch.tensor(tgt_indices).unsqueeze(0).to(device)

    with torch.no_grad():
        for _ in range(max_len):
            output = model(input_tensor, tgt_tensor)
            output = output.squeeze(0)
            next_token = output.argmax(dim=-1)[-1].item()
            tgt_indices.append(next_token)
            tgt_tensor = torch.tensor(tgt_indices).unsqueeze(0).to(device)
            if next_token == spa_word2idx['<eos>']:
                break

    return indices_to_sentence(tgt_indices, spa_idx2word)

### 7) Utilizando el modelo entrenado para traducir en del español al ingles un audio

In [37]:
import speech_recognition as sr

def audio_to_text(audio_data):
    recognizer = sr.Recognizer()
    
    # Convertir el audio grabado a un formato que SpeechRecognition pueda usar
    audio_data = sr.AudioData(audio_data.tobytes(), 44100, 1)
    
    try:
        # Reconocer el audio usando Google Web Speech API
        text = recognizer.recognize_google(audio_data, language='es-ES')  # Cambia 'es-ES' si usas otro idioma
        return text
    except sr.UnknownValueError:
        return "No se pudo entender el audio"
    except sr.RequestError as e:
        return f"No se pudo solicitar resultados; {e}"

# Asegúrate de integrar esta función en tu flujo principal

#### Probando grabacion audio

In [38]:
import sounddevice as sd
import numpy as np

def record_audio(duration=5, fs=44100):
    print("Grabando...")
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float32')
    sd.wait()  # Esperar a que termine la grabación
    return audio

def play_audio(audio):
    print("Reproduciendo...")
    sd.play(audio, 44100)
    sd.wait()  # Esperar a que termine de reproducir

# Graba audio
audio_data = record_audio(duration=5)
play_audio(audio_data)  # Reproduce el audio grabado

Grabando...
Reproduciendo...


## 8) Utilizando el modelo entrenado para la traducción de la voz en tiempo real

In [40]:
import torch
import sounddevice as sd
import numpy as np
import librosa
import time
from IPython.display import Audio, display


# Función para traducir texto
def translate_text(text):
    m_device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    translation = translate_sentence(model, text, eng_word2idx, spa_idx2word, max_len=MAX_SEQ_LEN, device=m_device)
    return translation

# Función para generar voz
def text_to_speech(text):
    # Aquí puedes utilizar una biblioteca de TTS
    # Este es un ejemplo usando gTTS
    from gtts import gTTS
    import os
    
    tts = gTTS(text, lang='en')
    tts.save("output.mp3")
    ##os.system("start output.mp3")  # Cambia 'start' según tu sistema operativo
                                     # Se comenta para que no se ejecute doble.

    # Reproducir audio directamente en Jupyter
    display(Audio("output.mp3", autoplay=True))

# Captura de audio en tiempo real
##def record_audio(duration=5, fs=44100):
def record_audio(duration=5, fs=16000):
    print("Grabando...")
    ##audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='float64')
    audio = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()  # Esperar a que termine la grabación
    return audio.flatten()
    #return audio

# Procesamiento en tiempo real
def real_time_translation():
    while True:
        audio_data = record_audio(duration=5)  # Graba 5 segundos de audio
        # Supongamos que tienes una función para convertir audio a texto
        text = audio_to_text(audio_data)  # Función ficticia que convierte audio a texto
        print(f"Texto detectado: {text}")

        translated_text = translate_text(text)
        print(f"Texto traducido: {translated_text}")

        text_to_speech(translated_text)
        time.sleep(1)  # Esperar un segundo antes de la próxima grabación

# Ejecutar
if __name__ == '__main__':
    # real_time_translation()
    try:
        real_time_translation()
    except KeyboardInterrupt:
        print("Detenido por el usuario.")
    except Exception as e:
        print(f"Ocurrió un error: {e}")

Grabando...
Texto detectado: hola hola hola hola
Texto traducido: allin pu hinaptinqa dios imanaqtintaq hinaptinqa


Grabando...
Texto detectado: Ah pues vamos a ver cómo nos hacen una cosa
Texto traducido: mana kamachikuy chhaynapi runakunaqa hap niran watamantapacha q wakintaq mana is hina mana is


Grabando...
Texto detectado: el chala jugando como papá
Texto traducido: huk karan se runan mana yanapawasunman


Grabando...
Detenido por el usuario.
